# 📈 Backtesting Strategy using Yahoo Finance Data (R)
This notebook loads stock ranking signals, fetches real stock prices from Yahoo Finance, 
processes the signals into a long/short portfolio, and backtests the strategy using R.

In [ ]:
install.packages(c('tidyverse', 'quantmod', 'xts', 'TTR'))

## 📌 Step 1: Load Required Libraries

In [ ]:
library(tidyverse)
library(quantmod)
library(xts)
library(TTR)

## 📌 Step 2: Load and Prepare Data

In [ ]:
# Load CSV file containing trend ranking data
df <- read.csv('https://storage.googleapis.com/antedata_fine/101/AllDat.csv')

# Filter timeseries where source is 'Aggregate'
df <- df[df$source == 'Aggregate', c('date', 'ticker', 'TrendRank')]

# Reshape data from long to wide format
df <- spread(df, key=ticker, value=TrendRank)

# Convert date column to Date format
df$date <- as.Date(df$date)
rownames(df) <- df$date
df <- df[,-1]  # Remove date column after setting row names

# Adjust signals to align with Fridays
index(df) <- index(df) + 4

# Display first few rows
head(df)

## 📌 Step 3: Retrieve Stock Prices from Yahoo Finance

In [ ]:
# Extract tickers
tickers <- colnames(df)

# Function to fetch stock prices
get_prices <- function(ticker_list, start_date, end_date) {
  prices <- list()
  for (ticker in ticker_list) {
    tryCatch({
      stock_data <- getSymbols(ticker, src='yahoo', from=start_date, to=end_date, auto.assign=FALSE)
      prices[[ticker]] <- Ad(stock_data)
    }, error=function(e){})
  }
  return(do.call(merge, prices))
}

# Fetch prices
prices <- get_prices(tickers, min(index(df)), max(index(df)))

# Handle missing data by forward-filling
prices <- na.locf(prices)

# Print first few rows
head(prices)

## 📌 Step 4: Process Stock Prices

In [ ]:
# Compute daily returns
returns <- ROC(prices, type='discrete')
returns <- na.omit(returns)

# Drop columns with excessive missing values
returns <- returns[, colSums(!is.na(returns)) > 0.9 * nrow(returns)]

# Display processed returns
head(returns)

## 📌 Step 5: Backtesting the Strategy

In [ ]:
SIG <- df

# Rank stocks and create long/short portfolio
SIG[] <- apply(SIG, 1, rank, ties.method='first')
SIG[SIG < (0.2 * ncol(SIG))] <- -0.03
SIG[SIG > (0.8 * ncol(SIG))] <- 0.03
SIG[SIG > 1] <- 0

# Harmonize columns of Signal and Returns
common_cols <- intersect(colnames(SIG), colnames(returns))
SIG <- SIG[, common_cols]
returns <- returns[, common_cols]

# Ensure data aligns correctly
SIG <- na.locf(SIG)

# Calculate backtest returns
OUT <- lag(SIG, 1) * returns

# Remove first 3 rows
OUT <- tail(OUT, -3)

# Display first few rows of output
head(OUT)

## 📌 Step 6: Visualizing the Strategy Performance

In [ ]:
library(ggplot2)
cum_returns <- cumsum(rowSums(OUT, na.rm=TRUE))
dates <- index(OUT)

ggplot(data.frame(Date=dates, CumulativeReturns=cum_returns), aes(x=Date, y=CumulativeReturns)) +
  geom_line(color='blue') +
  ggtitle('Backtest Returns with Yahoo Finance Prices') +
  xlab('Date') +
  ylab('Cumulative Returns') +
  theme_minimal()